# Script to convert between geojson (outputted by QGIS) and the json format required by labelme

In [1]:
import json
import os
from datetime import datetime

In [8]:
def geojson_to_labelme(geojson_path, image_path, image_height=None, image_width=None, label_name='waterhole'):
    """
    Convert GeoJSON point features to LabelMe JSON format.
    
    Args:
        geojson_path (str): Path to input GeoJSON file
        image_path (str): Path to the corresponding image file
        
    Returns:
        dict: LabelMe formatted JSON
    """
    # Read GeoJSON file
    with open(geojson_path, 'r') as f:
        geojson_data = json.load(f)
    
    # Initialize LabelMe JSON structure
    labelme_json = {
        "version": "5.6.1",
        "flags": {},
        "shapes": [],
        "imagePath": os.path.basename(image_path),
        "imageData": None,  # LabelMe stores base64 image data here, but we'll leave it empty
        "imageHeight": image_height,  
        "imageWidth": image_width
    }
    
    # Convert each GeoJSON feature to LabelMe shape
    for feature in geojson_data['features']:
        # Get coordinates (assuming Point geometry)
        coords = feature['geometry']['coordinates']
        
        # Create point annotation (LabelMe uses a small rectangle for points)
        point_size = 5  # Size of the point representation in pixels
        x, y = coords[0], coords[1]
        
        shape = {
            "label": label_name,
            # "label": feature['properties'].get('label', 'point'),  # Use 'label' property or default to 'point'
            "points": [
                [x - point_size, y - point_size],  # Top-left
                [x + point_size, y + point_size]   # Bottom-right
            ],
            "group_id": None,
            "shape_type": "rectangle",
            "flags": {}
        }
        
        labelme_json['shapes'].append(shape)
    
    # Add creation time
    labelme_json['timeStamp'] = datetime.now().isoformat()
    
    return labelme_json

def save_labelme_json(labelme_json, output_path):
    """Save the LabelMe JSON to file."""
    with open(output_path, 'w') as f:
        json.dump(labelme_json, f, indent=2)

In [9]:
# Example usage
if __name__ == "__main__":
    # Replace these paths with your actual file paths
    geojson_path = "images\labels\waterholes_holly_v1.geojson"
    image_path = "images\labels\20230428_example_SF.png"
    output_path = "images\labels\waterholes_holly_v1.json"
    
    # Convert and save
    labelme_json = geojson_to_labelme(geojson_path, image_path)
    save_labelme_json(labelme_json, output_path)